In [1]:
import pandas as pd
from datetime import datetime
import pandas as np

Información

- AccidentesPlantas: De momento no se va a usar. Hay grado de movilidad que se podria añadir.
- Balande hidroeléctrico: De momento no se va a usar.
- ConstantesVitales: Se extraer la tensión y la temperatura, pero en la mayoria de casos no hay datos. ----
- CurasVendajes: De momento no se va a usar.
- DatosIdentificativosAdministrativos:  !!!!!! IMPORTANTE ----
- DiagnósticosEnfermería: Mejor usar DiagnosticosEnfermeriaCodificados
- DiagnosticosEnfermeriaCodificados: De momento no se usa. 'gprobenf' y 'dprobenf' puede que sea importante. -----
- DiagnosticosMedicosCodificados: De momento no se usa. 'gdiagein', 'gcodidia',	'dnombdia'
- DietasViasAlimentacion: De momento  no se van a usar.
- InterconsultasSinDocumento: De momento no se va a usar.
- IntervencionesRealizadas: De momento no se va a usar.
- ManifestadoPor: De momento no se va a usar.
- Medidas: De momento no se va a usar.No info.
- OrdenesMedicas: De momento no se va a usar.
- PlanTipicoAsignado: De momento no se va a usar.No info.
- Prestaciones: De momento no se va a usar.
- ProcedimientosMedicosCodificados: De momento no se va a usar.
- RelacionadoCon: De momento no se va a usar.
- UlcerasPresion:De momento no se va a usar.


Faltaria esto:
- PrescripcionFarmacos: POR Terminar
- ValoracionesEnfermeria: POR Terminar con datos nuevos



### Carga de archivos

In [2]:
df = pd.read_csv('datos/DatosIdentificativosAdministrativos.csv')
df_constantes = pd.read_csv('datos/ConstantesVitales.csv')
df_glucemias = pd.read_csv('datos/GlucemiasGlucosurias.csv')
df_accidentes = pd.read_csv('datos/AccidentesPlanta.csv')
df_dietas = pd.read_csv('datos/DietasViasAlimentacion.csv')
df_antecedentes = pd.read_csv('datos/ValoracionesAntecedentesPersonales.csv')
df_valoracion_enfermeria = pd.read_csv('datos/ValoracionesEnfermeria.csv')
df_prescripcion_farmacos_1 = pd.read_csv('datos/PrescripcionFarmacos_1.csv')
df_prescripcion_farmacos_2 = pd.read_csv('datos/PrescripcionFarmacos_2.csv')


df_valoracion_medica_1 = pd.read_csv('datos/ValoracionesMedicas_1.csv')
df_valoracion_medica_2 = pd.read_csv('datos/ValoracionesMedicas_2.csv')
df_valoracion_medica_3 = pd.read_csv('datos/ValoracionesMedicas_3.csv')
df_valoracion_medica_4 = pd.read_csv('datos/ValoracionesMedicas_4.csv')
df_valoracion_medica_5 = pd.read_csv('datos/ValoracionesMedicas_5.csv')
df_valoracion_medica_6 = pd.read_csv('datos/ValoracionesMedicas_6.csv')
df_valoracion_medica_7 = pd.read_csv('datos/ValoracionesMedicas_7.csv')
df_valoracion_medica_8_1 = pd.read_csv('datos/ValoracionesMedicas_8.1.csv')
df_valoracion_medica_8_2 = pd.read_csv('datos/ValoracionesMedicas_8.2.csv')
df_valoracion_medica_9 = pd.read_csv('datos/ValoracionesMedicas_9.csv')
df_valoracion_medica_10 = pd.read_csv('datos/ValoracionesMedicas_10.csv')




/tmp/ipykernel_8697/2876111328.py:12: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_valoracion_medica_1 = pd.read_csv('datos/ValoracionesMedicas_1.csv')
/tmp/ipykernel_8697/2876111328.py:13: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_valoracion_medica_2 = pd.read_csv('datos/ValoracionesMedicas_2.csv')
/tmp/ipykernel_8697/2876111328.py:15: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_valoracion_medica_4 = pd.read_csv('datos/ValoracionesMedicas_4.csv')
/tmp/ipykernel_8697/2876111328.py:16: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_valoracion_medica_5 = pd.read_csv('datos/ValoracionesMedicas_5.csv')
/tmp/ipykernel_8697/2876111328.py:17: DtypeWarning: Columns (10,11,14) have mixed types. Specify dtype option on import or set low_memor

### Datos identificaticos

In [3]:
# Eliminar las columnas que no tienen información relevante
df = df.drop(columns=['odiagalt', 'gdiasec1', 'odiasec1', 'gdiasec2', 'odiasec2','itraslad', 'dmotalta','dsitalta','dnomserv','gserulti','gdiagini','gdiaging','dmotingr','gmotingr','gnumadme','ganoadme','dpoblaci','dnomprov','odiaging'	,'odiagini','iotrocen','itipingr' ])

In [4]:

#Nos quedamos solo con las fracturas de cadera. Empiezan todas por S72.
df = df[df['gdiagalt'].str.startswith('S72', na=False)]

# Eliminar filas duplicadas basadas en la columna 'gidenpac', conservando solo la primera aparición. Pasamos de 1078 a 1047 filas
df = df.drop_duplicates(subset='gidenpac', keep='first')

# Reemplazar 'H' por 0 y 'M' por 1 en la columna 'itipsexo'
df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})


# Columna ds_izq_der para saber que lado de la cadera es 
# ------------------------------------------------------------------------------------------
# Listas de códigos para "DERECHO" y "IZQUIERDO"
derecho_codes = ["001", "011", "051", "061", "091", "101", "111", "131", "141", "21", "301", "341", "391", "411", "421", "451", "491", "8X1", "91X"]
izquierdo_codes = ["002", "012", "042", "092", "102", "112", "115", "142", "22", "302", "425", "435", "452", "455", "492", "92X"]

# Categorizar la columna
def categorize_side(value):
    value = str(value)  # Convertir el valor a string por seguridad
    if any(code in value for code in derecho_codes):
        return 1
    elif any(code in value for code in izquierdo_codes):
        return 0
    else:
        return None  # Marcar para eliminar

# Crear la nueva columna 'ds_izq_der' basada en la función de categorización
df['ds_izq_der'] = df['gdiagalt'].apply(categorize_side)

# Eliminar las filas donde 'ds_izq_der' es None
df = df.dropna(subset=['ds_izq_der'])   # Solo perdemos 1
# ------------------------------------------------------------------------------------------

# Columna turnos de trabajo
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fllegada' a formato de fecha y hora
df['fllegada'] = pd.to_datetime(df['fllegada'])

# Función para asignar el turno en función de la hora
def asignar_turno(hora):
    if 8 <= hora < 15:      #Mañana
        return 0
    elif 15 <= hora < 22:   # Tarde
        return 1
    else:                   # Noche
        return 2

# Crear una nueva columna 'ds_turno' basada en la hora de 'fllegada'
df['ds_turno'] = df['fllegada'].dt.hour.apply(asignar_turno)
# ------------------------------------------------------------------------------------------


# Columna edad
# ------------------------------------------------------------------------------------------
# Convertir la columna 'fnacipac' a formato de fecha
df['fnacipac'] = pd.to_datetime(df['fnacipac'])

# Función para calcular la edad
def calcular_edad(fecha_nacimiento):
    hoy = datetime.now()
    edad = hoy.year - fecha_nacimiento.year - ((hoy.month, hoy.day) < (fecha_nacimiento.month, fecha_nacimiento.day))
    return edad

# Crear la nueva columna 'ds_edad' calculando la edad para cada fila
df['ds_edad'] = df['fnacipac'].apply(calcular_edad)
# ------------------------------------------------------------------------------------------

# Columnas de días estancia, preopertorio y postoperatorio
# ------------------------------------------------------------------------------------------
# Convertir las columnas de fecha a formato datetime, sin considerar las horas
df['fllegada'] = pd.to_datetime(df['fllegada']).dt.normalize()
df['faltplan'] = pd.to_datetime(df['faltplan']).dt.normalize()
df['finterve'] = pd.to_datetime(df['finterve']).dt.normalize()

# Calcular los días de estancia total (faltplan - fllegada)
df['ds_estancia'] = (df['faltplan'] - df['fllegada']).dt.days

# Calcular los días de preoperatorio incluyendo el día de intervención
df['ds_pre_oper'] = (df['finterve'] - df['fllegada']).dt.days

# Calcular los días de postoperatorio incluyendo el día de intervención
df['ds_post_oper'] = (df['faltplan'] - df['finterve']).dt.days
# ------------------------------------------------------------------------------------------

# Crear la columna 'ds_vivo_alta' basada en la condición de 'gsitalta'
df['ds_vivo_alta'] = df['gsitalta'].apply(lambda x: 0 if x == 7 else 1)

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ireingre'
df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})

# Crear la columna 'ds_dia_semana_llegada'(1 para lunes, 7 para domingo)
df['ds_dia_semana_llegada'] = df['fllegada'].dt.dayofweek + 1

# Crear la columna 'ds_mes_llegada'  
df['ds_mes_llegada'] = df['fllegada'].dt.month

# Eliminar filas de códigos (gdiagalt) que apenas tengamos info
codigos_permitidos = ['S72.141A', 'S72.012A', 'S72.142A', 'S72.011A', 
                      'S72.002A', 'S72.001A', 'S72.102A', 'S72.111A', 'S72.102A','S72.101A' 'S72.111A', 'S72.112A' ,'S72.092A', 'S72.091A' ]
# Filtrar el DataFrame para quedarse solo con las filas cuyo valor en 'gdiagalt' esté en la lista
df = df[df['gdiagalt'].isin(codigos_permitidos)]

# Crear columa tipo_fractura. 
replace_dict = {
    'S72.011A': 1, # Intracapsular --> Intracapsular no desplazada
    'S72.012A': 1,  
    'S72.141A': 2, # Intertrocantérea --> Intracapsular desplazada
    'S72.142A': 2, 
    'S72.111A': 4, # Pertrocantérea
    'S72.112A': 4, 
    'S72.101A': 4, 
    'S72.102A': 4, 
    'S72.091A': 5, # Otras
    'S72.092A': 5, 
    'S72.001A': 5, 
    'S72.002A': 5
}

# Mapear los valores de la columna 'gdiagalt' al nuevo valor en 'tipo_fractura'
df['ds_tipo_fractura'] = df['gdiagalt'].map(replace_dict)

# Obtenemos si viven en el centro o en las afueras
codigos_centro = [24001, 24002, 24003, 24004, 24005, 24006, 24007, 24008, 24009, 24010, 24012, 24070, 24071, 24080, ]  #Códigos del centro
df['ds_centro_afueras'] = df['gcodipos'].apply(lambda x: 1 if x in codigos_centro else 0)


# Eliminar columnas que ya no necesitamos
df = df.drop(columns=['faltplan'])
df = df.drop(columns=['fnacipac'])
df = df.drop(columns=['gprovinc'])
df = df.drop(columns=['fllegada'])
df = df.drop(columns=['finterve'])

# Pasamos a enteros y los vacios los dejamos en -999
df['gcodipos'] = df['gcodipos'].fillna(-999).astype(int)
df['gpoblaci'] = df['gpoblaci'].fillna(-999).astype(int)
df['ds_izq_der'] = df['ds_izq_der'].fillna(-999).astype(int)
df['ds_pre_oper'] = df['ds_pre_oper'].fillna(-999).astype(int)
df['ds_post_oper'] = df['ds_post_oper'].fillna(-999).astype(int)


/tmp/ipykernel_8697/2102882064.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['itipsexo'] = df['itipsexo'].replace({'H': 0, 'M': 1})
/tmp/ipykernel_8697/2102882064.py:89: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ireingre'] = df['ireingre'].replace({'N': 0, 'S': 1})


In [5]:
df

,gidenpac,itipsexo,gcodipos,gpoblaci,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,ds_edad,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada,ds_tipo_fractura,ds_centro_afueras
16,MRMR192402260,0,24006,242127,0,7,3,S72.142A,0,1,100,5,2,3,0,1,9,2,1
32,GRMN192606210,0,24328,241396,0,1,1,S72.012A,0,1,98,19,8,11,1,4,3,1,0
111,BLGR192807170,0,24006,240896,0,2,1,S72.012A,0,1,96,7,3,4,1,5,2,1,1
165,LFLL192212310,0,24240,241572,0,2,1,S72.141A,1,1,101,8,4,4,1,4,6,2,0
237,DRCL192405020,0,24600,241149,0,2,1,S72.012A,0,2,100,17,9,8,1,7,4,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42957,BZMR193509440,1,24010,240896,0,2,1,S72.012A,0,2,89,8,3,5,1,2,5,1,1
42958,LVGR193509470,1,24001,240896,0,2,1,S72.012A,0,1,89,8,1,7,1,2,4,1,1
42962,RDDM193509620,1,24358,240358,0,2,1,S72.141A,1,1,89,15,1,14,1,4,5,2,0
43035,CBCS194203680,1,24240,241572,0,2,1,S72.141A,1,1,82,16,4,12,1,1,4,2,0


### Accidentes


In [6]:
# Filtrar df_accidentes para seleccionar solo las columnas relevantes
df_accidentes = df_accidentes[['gidenpac', 'ifamilia', 'gmovilid']]

# Realizar el merge con df, añadiendo las columnas de 'ifamilia' donde coincida 'gidenpac'
df = pd.merge(df, df_accidentes, on='gidenpac', how='left')

# Reemplazar 'N' por 0 y 'S' por 1 en la columna 'ifamilia'
df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})

# Cambiar el nombre de la columna 'ifamilia' a 'vive_solo'
df = df.rename(columns={'ifamilia': 'vive_solo'})

# Renombrar 'gmovilid' como 'movilidad_accidentes'
df = df.rename(columns={'gmovilid': 'movilidad_accidentes'})

# Pasar a entero y añadimos -999 en los vacios
df['vive_solo'] = df['vive_solo'].fillna(-999).astype(int)
df['movilidad_accidentes'] = df['movilidad_accidentes'].fillna(-999).astype(int)


/tmp/ipykernel_8697/3593642863.py:8: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['ifamilia'] = df['ifamilia'].replace({'N': 0, 'S': 1})


In [7]:
df

,gidenpac,itipsexo,gcodipos,gpoblaci,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,...,ds_estancia,ds_pre_oper,ds_post_oper,ds_vivo_alta,ds_dia_semana_llegada,ds_mes_llegada,ds_tipo_fractura,ds_centro_afueras,vive_solo,movilidad_accidentes
0,MRMR192402260,0,24006,242127,0,7,3,S72.142A,0,1,...,5,2,3,0,1,9,2,1,-999,-999
1,GRMN192606210,0,24328,241396,0,1,1,S72.012A,0,1,...,19,8,11,1,4,3,1,0,-999,-999
2,BLGR192807170,0,24006,240896,0,2,1,S72.012A,0,1,...,7,3,4,1,5,2,1,1,-999,-999
3,LFLL192212310,0,24240,241572,0,2,1,S72.141A,1,1,...,8,4,4,1,4,6,2,0,-999,-999
4,DRCL192405020,0,24600,241149,0,2,1,S72.012A,0,2,...,17,9,8,1,7,4,1,0,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,240896,0,2,1,S72.012A,0,2,...,8,3,5,1,2,5,1,1,-999,-999
976,LVGR193509470,1,24001,240896,0,2,1,S72.012A,0,1,...,8,1,7,1,2,4,1,1,-999,-999
977,RDDM193509620,1,24358,240358,0,2,1,S72.141A,1,1,...,15,1,14,1,4,5,2,0,-999,-999
978,CBCS194203680,1,24240,241572,0,2,1,S72.141A,1,1,...,16,4,12,1,1,4,2,0,0,2


### Constantes vitales

In [8]:
#La mayoria están vacios.

# Seleccionar solo las columnas necesarias de df_constantes y eliminar duplicados tomando el primer valor no nulo
df_constantes = df_constantes[['gidenpac', 'ntensmin', 'ntensmax', 'ntempera', 'nsatuoxi']].dropna(how='all')
df_constantes = df_constantes.groupby('gidenpac').first().reset_index()

# Realizar el merge con df, añadiendo las constantes vitales
df = pd.merge(df, df_constantes, on='gidenpac', how='left')

# Pasamos a int y añadimos -999 en los vacios. En la temperatura dejamos float
df['ntensmin'] = df['ntensmin'].fillna(-999).astype(int)
df['ntensmax'] = df['ntensmax'].fillna(-999).astype(int)
df['ntempera'] = df['ntempera'].fillna(-999).astype(float)
df['nsatuoxi'] = df['nsatuoxi'].fillna(-999).astype(int)

In [9]:
df

,gidenpac,itipsexo,gcodipos,gpoblaci,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,...,ds_dia_semana_llegada,ds_mes_llegada,ds_tipo_fractura,ds_centro_afueras,vive_solo,movilidad_accidentes,ntensmin,ntensmax,ntempera,nsatuoxi
0,MRMR192402260,0,24006,242127,0,7,3,S72.142A,0,1,...,1,9,2,1,-999,-999,-999,-999,-999.0,-999
1,GRMN192606210,0,24328,241396,0,1,1,S72.012A,0,1,...,4,3,1,0,-999,-999,-999,-999,-999.0,-999
2,BLGR192807170,0,24006,240896,0,2,1,S72.012A,0,1,...,5,2,1,1,-999,-999,-999,-999,-999.0,-999
3,LFLL192212310,0,24240,241572,0,2,1,S72.141A,1,1,...,4,6,2,0,-999,-999,-999,-999,-999.0,-999
4,DRCL192405020,0,24600,241149,0,2,1,S72.012A,0,2,...,7,4,1,0,-999,-999,-999,-999,-999.0,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,240896,0,2,1,S72.012A,0,2,...,2,5,1,1,-999,-999,-999,-999,-999.0,-999
976,LVGR193509470,1,24001,240896,0,2,1,S72.012A,0,1,...,2,4,1,1,-999,-999,-999,-999,-999.0,-999
977,RDDM193509620,1,24358,240358,0,2,1,S72.141A,1,1,...,4,5,2,0,-999,-999,-999,-999,-999.0,-999
978,CBCS194203680,1,24240,241572,0,2,1,S72.141A,1,1,...,1,4,2,0,0,2,-999,-999,-999.0,-999


### Antecedentes


In [10]:
# Seleccionar solo las columnas necesarias de 'df_antecedentes' y crear una copia
df_ant = df_antecedentes[['gidenpac', 'dconclin', 'vbivalor']].copy()

# Crear columnas en 'df_ant' basadas en el contenido de 'dconclin'
df_ant['ds_HTA'] = df_ant["dconclin"].str.contains("HTA", na=False).astype(int)
df_ant['ds_alergia_medicamentosa'] = df_ant["dconclin"].str.contains("Alergia medicamentosa").astype(int)
df_ant['ds_alergia_alimenticia'] = df_ant["dconclin"].str.contains("Alergia alimenticia").astype(int)
df_ant['ds_diabetes'] = df_ant["dconclin"].str.contains("Diabetes Mellitus").astype(int)
df_ant['ds_otras_alergias'] = df_ant["dconclin"].str.contains("Otras alergias").astype(int)

# Actualizar valores en columnas de antecedentes usando 'vbivalor' si no es nulo
columns_to_update = ['ds_HTA', 'ds_alergia_medicamentosa', 'ds_alergia_alimenticia', 'ds_otras_alergias']
for col in columns_to_update:
    df_ant[col] = df_ant.apply(lambda x: x['vbivalor'] if pd.notnull(x['vbivalor']) and x[col] == 1 else x[col], axis=1)

# Reemplazar valores específicos ('N' -> 0, 'S' -> 1, 'A' -> 1) en todo el DataFrame
df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)

# Eliminar las columnas 'dconclin' y 'vbivalor' si ya no son necesarias
df_ant.drop(columns=['dconclin', 'vbivalor'], inplace=True)

# Eliminar duplicados en el DataFrame (si aún no se ha hecho)
df_ant = df_ant.drop_duplicates()

# Agrupar por 'gidenpac' y usar 'max' para que cualquier 1 en una columna mantenga el valor como 1
df_ant = df_ant.groupby('gidenpac', as_index=False).max()

# Realizar un merge entre df y df_ant sin sufijos
df = pd.merge(df, df_ant, on='gidenpac', how='left')

# Asignar valores de df_ant solo si son 1
for col in columns_to_update:
    df[col] = df.apply(lambda x: x[col] if pd.notnull(x[col]) and x[col] == 1 else np.nan if pd.isna(x[col]) else x[col], axis=1)


/tmp/ipykernel_8697/2776989086.py:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_ant.replace({'N': 0, 'S': 1, 'A': 1}, inplace=True)


In [11]:
df_ant

,gidenpac,ds_HTA,ds_alergia_medicamentosa,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias
0,BBMR195406141,1,0,0,1,0
1,BBRV196104300,0,0,0,0,0
2,BBXX193502600,0,0,0,0,0
3,BCCB200202020,0,0,0,0,0
4,BCCN193804700,0,0,0,1,0
...,...,...,...,...,...,...
33000,ZTLN196202550,0,0,0,1,0
33001,ZYML195212020,0,0,0,0,0
33002,ZZBD198908430,0,0,0,0,0
33003,ZZXX197406420,0,0,0,0,0


### Valoracion Enfermeria

In [12]:
import pandas as pd
import numpy as np

# Seleccionar solo las columnas necesarias de 'df_valoracion_enfermeria'
df_valoracion_enfermeria = df_valoracion_enfermeria[['gidenpac', 'dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor']].copy()

# Asegurarse de que 'ncodtabu', 'nvalncon', 'ovallcon', y 'vbivalor' sean numéricos
df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']] = df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].apply(pd.to_numeric, errors='coerce')

# Definir los valores de 'dconclin' que se convertirán en nuevas columnas
patterns = {
    '_movilidad': "_Movilidad",
    'movilidad': "Movilidad",
    'lugar_residencia': "G Lugar de Residencia",
    'lugar_procedencia': "G Procedencia",
    'destino_alta': "G Destino Alta",
    'Barthel': "Resultado Indice de Barthel",
    'barthel_alta': "Resultado Indice de Barthel al Alta",
    'braden': "Resultado Escala de Braden",
    'riesgo_caida': "Resultado Escala Riesgo Caidas"
}

# Crear una expresión regular unida para filtrar las filas deseadas
regex_pattern = "|".join(patterns.values())

# Filtrar el DataFrame para conservar solo las filas que coinciden con el patrón
df_valoracion_enfermeria = df_valoracion_enfermeria[df_valoracion_enfermeria['dconclin'].str.contains(regex_pattern, na=False)]

# Iterar sobre el diccionario para crear las columnas basadas en el valor máximo entre las columnas especificadas
for new_col, pattern in patterns.items():
    # Crear la nueva columna con el valor máximo de las columnas seleccionadas donde 'dconclin' coincide con el patrón
    df_valoracion_enfermeria[new_col] = np.where(
        df_valoracion_enfermeria['dconclin'].str.contains(pattern, na=False),
        df_valoracion_enfermeria[['ncodtabu', 'nvalncon', 'ovallcon', 'vbivalor']].max(axis=1),
        np.nan
    )

# Agrupar por 'gidenpac' y mantener el valor máximo en cada columna para eliminar duplicados
df_valoracion_enfermeria = df_valoracion_enfermeria.groupby('gidenpac', as_index=False).max()

# Eliminar las columnas no deseadas del DataFrame
df_valoracion_enfermeria = df_valoracion_enfermeria.drop(columns=['dconclin', 'ncodtabu', 'nsecuval', 'nvalncon', 'ovallcon', 'vbivalor', '_movilidad','barthel_alta'])

# Combinar df y df_valoracion_enfermeria por la columna 'gidenpac'
df= pd.merge(df, df_valoracion_enfermeria, on='gidenpac', how='left')

# Convertimos a enteros y los Nan los pasamos a -999
df['movilidad'] = df['movilidad'].fillna(-999).astype(int)
df['lugar_residencia'] = df['lugar_residencia'].fillna(-999).astype(int)
df['lugar_procedencia'] = df['lugar_procedencia'].fillna(-999).astype(int)
df['destino_alta'] = df['destino_alta'].fillna(-999).astype(int)
df['Barthel'] = df['Barthel'].fillna(-999).astype(int)
df['braden'] = df['braden'].fillna(-999).astype(int)
df['riesgo_caida'] = df['riesgo_caida'].fillna(-999).astype(int)


In [13]:
df_valoracion_enfermeria

,gidenpac,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,BRGR193406480,2.0,NaN,NaN,NaN,20.0,15.0,6.0
1,CRLN197704540,3.0,NaN,NaN,NaN,NaN,18.0,2.0
2,CSBR193504540,2.0,NaN,NaN,NaN,0.0,12.0,7.0
3,CSPR193305700,3.0,NaN,NaN,NaN,80.0,21.0,5.0
4,FDFD192912290,2.0,NaN,NaN,NaN,35.0,NaN,7.0
5,FRDL195208630,2.0,NaN,NaN,NaN,NaN,13.0,7.0
6,FRDP192501100,3.0,NaN,NaN,NaN,55.0,16.0,5.0
7,FRGR193404470,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8,FRGT192606121,3.0,NaN,NaN,NaN,40.0,17.0,3.0
9,FRHR194705480,2.0,NaN,NaN,NaN,5.0,15.0,4.0


In [14]:
df

,gidenpac,itipsexo,gcodipos,gpoblaci,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,...,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,MRMR192402260,0,24006,242127,0,7,3,S72.142A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
1,GRMN192606210,0,24328,241396,0,1,1,S72.012A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
2,BLGR192807170,0,24006,240896,0,2,1,S72.012A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
3,LFLL192212310,0,24240,241572,0,2,1,S72.141A,1,1,...,1,1,0,-999,-999,-999,-999,-999,-999,-999
4,DRCL192405020,0,24600,241149,0,2,1,S72.012A,0,2,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,240896,0,2,1,S72.012A,0,2,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
976,LVGR193509470,1,24001,240896,0,2,1,S72.012A,0,1,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
977,RDDM193509620,1,24358,240358,0,2,1,S72.141A,1,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
978,CBCS194203680,1,24240,241572,0,2,1,S72.141A,1,1,...,0,1,0,-999,-999,-999,-999,-999,-999,-999


### Glucemias

In [15]:
# Pocos datos

# Crear un conjunto con los valores de gidenpac en df_glucemias
glucemias_pacientes = set(df_glucemias['gidenpac'].dropna())

# Crear la columna 'ds_diabetes' solo en las casillas vacías. Añadimos info a la casilla ds_diabetes
df['ds_diabetes'] = df.apply(lambda row: 1 if pd.isna(row['ds_diabetes']) and row['gidenpac'] in glucemias_pacientes 
                             else (0 if pd.isna(row['ds_diabetes']) else row['ds_diabetes']), axis=1)


### Valoraciones Medicas

In [16]:
# Definir los códigos de 'gcodconc' que queremos filtrar para los diagnosticos
codigos = [1368, 1369, 1370, 1371, 1372, 1373, 1374, 1742, 1743, 1750, 1813, 1814, 1815]

# Listado de archivos a procesar
archivos = [
    'datos/ValoracionesMedicas_1.csv',
    'datos/ValoracionesMedicas_2.csv',
    'datos/ValoracionesMedicas_3.csv',
    'datos/ValoracionesMedicas_4.csv',
    'datos/ValoracionesMedicas_5.csv',
    'datos/ValoracionesMedicas_6.csv',
    'datos/ValoracionesMedicas_7.csv',
    'datos/ValoracionesMedicas_8.1.csv',
    'datos/ValoracionesMedicas_8.2.csv',
    'datos/ValoracionesMedicas_9.csv',
    'datos/ValoracionesMedicas_10.csv'
]

# Lista para almacenar los DataFrames filtrados
dataframes_filtrados = []

# Iterar sobre cada archivo
for archivo in archivos:
    # Cargar el archivo
    df_v_med = pd.read_csv(archivo)
    
    # Filtrar las filas donde 'gcodconc' esté en la lista de códigos
    df_filtrado = df_v_med[df_v_med['gcodconc'].isin(codigos)]
    
    # Seleccionar solo las columnas 'gidenpac', 'gcodconc' y 'ovallcon'
    df_filtrado = df_filtrado[['gidenpac', 'gcodconc', 'ovallcon']]
    
    # Añadir el DataFrame filtrado a la lista
    dataframes_filtrados.append(df_filtrado)

# Concatenar todos los DataFrames filtrados en uno solo
df_val_medicas_combined = pd.concat(dataframes_filtrados, ignore_index=True)


# Guardar el resultado en un archivo CSV
df_val_medicas_combined.to_csv('DF_VAL_MEDICAS_COMBINED.csv', index=False)

conteo_codigos = df_val_medicas_combined['ovallcon'].value_counts()



/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_v_med = pd.read_csv(archivo)
/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_v_med = pd.read_csv(archivo)
/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_v_med = pd.read_csv(archivo)
/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df_v_med = pd.read_csv(archivo)
/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df_v_med = pd.read_csv(archivo)
/tmp/ipykernel_8697/493482408.py:25: DtypeWarning: Columns (10,11) have mixed types. Specify dtype option on import or set

In [17]:
# Crear la nueva columna 'ds_HTA' para detectar 'HTA' en cualquier parte del texto
df_val_medicas_combined['ds_HTA'] = df_val_medicas_combined['ovallcon'].str.contains('HTA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_ITU'] = df_val_medicas_combined['ovallcon'].str.contains('ITU', case=False, na=False).astype(int)
df_val_medicas_combined['ds_acido_folico'] = df_val_medicas_combined['ovallcon'].str.contains('ACIDO FOLICO', case=False, na=False).astype(int)
df_val_medicas_combined['ds_anemia'] = df_val_medicas_combined['ovallcon'].str.contains('ANEMIA', case=False, na=False).astype(int)
df_val_medicas_combined['ds_vitamina_d'] = df_val_medicas_combined['ovallcon'].str.contains('VITAMINA D', case=False, na=False).astype(int)
df_val_medicas_combined['ds_obesidad'] = df_val_medicas_combined['ovallcon'].str.contains('OBESIDAD', case=False, na=False).astype(int)
df_val_medicas_combined['ds_osteoporosis'] = df_val_medicas_combined['ovallcon'].str.contains('OSTEOPOROSIS', case=False, na=False).astype(int)


# Insuficiencia respiratoria
# ___________________________________________________________
respiratory_terms = [
    'INSUFICIENCIA RESPIRATORIA AGUDA', 'INSUFICIENCIA RESPIRATORIA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL', 'INSUFICIENCIA RESPIRATORIA CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL', 'INSUFICIENCIA RESPIRATORIA 2RIA', 
    'INSUFICIENCIA RESPIRATORIA GLOBAL', 'INSUFICIENCIA RESPIRATORIA AGUDA SECUNDARIA', 
    'INSUFICIENCIA RESPIRATORIA PARCIAL CRÓNICA AGUDIZADA', 
    'INSUFICIENCIA RESPIRATORIA AGUDA PARCIAL RESUELTA', 'EPOC AGUDIZADO', 'EPOC', 'EPOC REAGUDIZADO'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(respiratory_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Diabetes
# ___________________________________________________________
diabetes_terms = [
    'DM TIPO 2 ', 'DM TIPO II', 'DIABETES MELLITUS', 'DM-II',  'DIABETES MELLITUS TIPO II','DIABETES'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(diabetes_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_diabetes'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Infección respiratoria
# ___________________________________________________________
infeccion_respiratoria_terms = [
   'INFECCIÓN RESPIRATORIA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS',  'INFECCIÓN RESPIRATORIA DE VÍAS INFERIORES', 'INFECCION RESPIRATORIA', 'INFECCION RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA AGUDA', 'INFECCIÓN RESPIRATORIA DE VÍAS BAJAS NO CONDENSANTE'
]

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(infeccion_respiratoria_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_infeccion_respiratoria'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia Cardiaca
# ___________________________________________________________
insuficiencia_cardiaca_terms = [
'INSUFICIENCIA CARDIACA', 'INSUFICIENCIA CARDIACA DESCOMPENSADA',  'INSUFICIENCIA RESPIRATORIA CRÓNICA', 'INSUFICIENCIA CARDIACA CONGESTIVA',  'I. CARDIACA', 'INSUFICIENCIA CARDIACA CONGESTIVA DESCOMPENSADA', 'INSUFICIENCIA CARDÍACA', 'INSUFICIENCIA CARDIACA CRÓNICA DESCOMPENSADA',  'INSUFICIENCIA CARDIACA CONGESTIVA AGUDA', 'ICC AGUDA', 'CARDIOPATIA ISQUEMICA',  'CARDIOPATÍA ISQUÉMICA', 'ICC', 'ICC DESCOMPENSADA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_cardiaca_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_cardiaca'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Deterioro cognitivo
# ___________________________________________________________
deterioro_cognitivo_terms = [
	'DETERIORO COGNITIVO', 'DETERIORO COGNITIVO', 'DEMENCIA']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(deterioro_cognitivo_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_deterioro_cognitivo'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Exitus
# ___________________________________________________________
exitus_terms = [
    'EXITUS', 'ÉXITUS']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(exitus_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_exitus'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________

# Insuficiencia renal
# ___________________________________________________________
insuficiencia_renal_terms = [
    'I. RENAL', 'INSUFICIENCIA RENAL', 'ENFERMEDAD RENAL CRÓNICA', 'INSUFICIENCIA RENAL AGUDA', 'INSUFICIENCIA RENAL CRÓNICA AGUDIZADA', 'INSUFICIENCIA RENAL CRONICA', 'IRC', 'ERC REAGUDIZADA', 'ERC AGUDIZADA', 'ERC']

# Crear una expresión regular que detecte cualquiera de los términos
pattern = '|'.join(insuficiencia_renal_terms)

# Crear la nueva columna 'ds_respiratoria' con 1 si encuentra cualquier término
df_val_medicas_combined['ds_insuficiencia_renal'] = df_val_medicas_combined['ovallcon'].str.contains(pattern, case=False, na=False).astype(int)
# ___________________________________________________________


# Eliminamos las columnas que ya no necesitamos
df_filtrado = df_val_medicas_combined.drop(columns=['gcodconc', 'ovallcon'])

# Agrupar por 'gidenpac' y aplicar max() para cada columna
df_val_medicas_procesado = df_filtrado.groupby('gidenpac').max().reset_index()


df_val_medicas_procesado.to_csv('DF_VAL_MEDICAS_PROCESED.csv', index=False)



In [18]:
df

,gidenpac,itipsexo,gcodipos,gpoblaci,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,...,ds_alergia_alimenticia,ds_diabetes,ds_otras_alergias,movilidad,lugar_residencia,lugar_procedencia,destino_alta,Barthel,braden,riesgo_caida
0,MRMR192402260,0,24006,242127,0,7,3,S72.142A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
1,GRMN192606210,0,24328,241396,0,1,1,S72.012A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
2,BLGR192807170,0,24006,240896,0,2,1,S72.012A,0,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
3,LFLL192212310,0,24240,241572,0,2,1,S72.141A,1,1,...,1,1,0,-999,-999,-999,-999,-999,-999,-999
4,DRCL192405020,0,24600,241149,0,2,1,S72.012A,0,2,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,240896,0,2,1,S72.012A,0,2,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
976,LVGR193509470,1,24001,240896,0,2,1,S72.012A,0,1,...,0,1,0,-999,-999,-999,-999,-999,-999,-999
977,RDDM193509620,1,24358,240358,0,2,1,S72.141A,1,1,...,0,0,0,-999,-999,-999,-999,-999,-999,-999
978,CBCS194203680,1,24240,241572,0,2,1,S72.141A,1,1,...,0,1,0,-999,-999,-999,-999,-999,-999,-999


### Filtrado de los datos finales

Unir el DataFrame principal (df) con el de valoraciones medicas ya filtrado y procesado (df_val_medicas_procesado)

In [19]:
# Unir df y df_val_medicas_procesado por la columna 'gidenpac'
df = df.merge(df_val_medicas_procesado, on='gidenpac', how='left')
# Reemplazar valores NaN o nulos por -999
df.fillna(-999, inplace=True)

# Convertimos a enteros y los Nan los pasamos a -999
df['ds_vitamina_d'] = df['ds_vitamina_d'].fillna(-999).astype(int)
df['ds_obesidad'] = df['ds_obesidad'].fillna(-999).astype(int)
df['ds_osteoporosis'] = df['ds_osteoporosis'].fillna(-999).astype(int)
df['ds_insuficiencia_respiratoria'] = df['ds_insuficiencia_respiratoria'].fillna(-999).astype(int)
df['ds_diabetes_y'] = df['ds_diabetes_y'].fillna(-999).astype(int)
df['ds_infeccion_respiratoria'] = df['ds_infeccion_respiratoria'].fillna(-999).astype(int)
df['ds_insuficiencia_cardiaca'] = df['ds_insuficiencia_cardiaca'].fillna(-999).astype(int)
df['ds_deterioro_cognitivo'] = df['ds_deterioro_cognitivo'].fillna(-999).astype(int)
df['ds_insuficiencia_renal'] = df['ds_insuficiencia_renal'].fillna(-999).astype(int)
df['ds_HTA_y'] = df['ds_HTA_y'].fillna(-999).astype(int)
df['ds_ITU'] = df['ds_ITU'].fillna(-999).astype(int)
df['ds_acido_folico'] = df['ds_acido_folico'].fillna(-999).astype(int)
df['ds_anemia'] = df['ds_anemia'].fillna(-999).astype(int)

Vamos a combinar las 2 columnas de HTA.
- HTA: Tenemos 2. X e Y. 
	- X viene de antecedentes
	- Y de valoraciones médicas
- Podemos combinar las 2, ya que es más probable que no pongan la info, a poner una info falsa


In [20]:
df['ds_HTA'] = df[['ds_HTA_x', 'ds_HTA_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_HTA_x', 'ds_HTA_y'], inplace=True)


Hacemos lo mismo con la diabetes


In [21]:
df['ds_diabetes'] = df[['ds_diabetes_x', 'ds_diabetes_y']].max(axis=1)

# Eliminar las columnas originales
df.drop(columns=['ds_diabetes_x', 'ds_diabetes_y'], inplace=True)

Eliminar columnas que no nos van a servir


In [22]:
df.drop(columns=['ds_exitus'], inplace=True)    # Tenemos la varibale vivo al alta (ds_vivo_alta)
df.drop(columns=['gpoblaci'], inplace=True)  # La elimino porque tengo el codigo postal (gcodpos)



Eliminamos filas que les falten datos importantes

In [23]:
df = df[df['ds_pre_oper'] != -999]  # Eliminamos filas que no tengan los dias de pre o post operacion
df = df[df['ds_post_oper'] != -999]

# Si las valoraciones médicas no tienen info sobre el paciente, lo eliminamos
columnas_a_verificar = [
    'ds_ITU', 'ds_acido_folico', 'ds_anemia', 'ds_vitamina_d', 
    'ds_obesidad', 'ds_osteoporosis', 'ds_insuficiencia_respiratoria', 
    'ds_infeccion_respiratoria', 'ds_insuficiencia_cardiaca', 
    'ds_deterioro_cognitivo', 'ds_insuficiencia_renal'
]

df = df[~df[columnas_a_verificar].isin([-999]).any(axis=1)]


In [24]:
df

,gidenpac,itipsexo,gcodipos,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,ds_edad,...,ds_vitamina_d,ds_obesidad,ds_osteoporosis,ds_insuficiencia_respiratoria,ds_infeccion_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes
1,GRMN192606210,0,24328,0,1,1,S72.012A,0,1,98,...,0,0,0,1,1,1,0,1,0,0
2,BLGR192807170,0,24006,0,2,1,S72.012A,0,1,96,...,0,0,0,0,0,0,1,0,0,0
3,LFLL192212310,0,24240,0,2,1,S72.141A,1,1,101,...,1,0,0,0,0,0,1,0,0,1
4,DRCL192405020,0,24600,0,2,1,S72.012A,0,2,100,...,0,0,0,1,1,1,0,0,0,1
5,MNHR191911290,0,24007,0,2,1,S72.012A,0,2,104,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,0,2,1,S72.012A,0,2,89,...,0,0,0,0,0,0,0,0,1,1
976,LVGR193509470,1,24001,0,2,1,S72.012A,0,1,89,...,0,0,0,0,0,1,1,1,1,1
977,RDDM193509620,1,24358,0,2,1,S72.141A,1,1,89,...,0,0,0,0,0,0,0,0,0,0
978,CBCS194203680,1,24240,0,2,1,S72.141A,1,1,82,...,1,0,0,0,0,0,0,0,0,1


### Prescripción Farmacos

In [25]:
# Medicamentos prescitos
# Se podria usar? 

# Seleccionar solo las columnas 'gidenpac' y 'tarasist'
df_prescripcion_farmacos_1 = df_prescripcion_farmacos_1[['gidenpac', 'tarasist']]
df_prescripcion_farmacos_2 = df_prescripcion_farmacos_2[['gidenpac', 'tarasist']]

# Combinar los dos DataFrames
df_combined = pd.concat([df_prescripcion_farmacos_1, df_prescripcion_farmacos_2], ignore_index=True)

# Agrupar por 'gidenpac' y combinar los medicamentos en una lista
df_medicamentos = df_combined.groupby('gidenpac')['tarasist'].apply(lambda x: ', '.join(x.dropna().astype(str))).reset_index()

# Verificar el resultado
print(df_medicamentos.head())

df_medicamentos.to_csv('medicamentos.csv')


        gidenpac                                           tarasist
0  BRGR193406480  PARACETAMOL 10 MG/ML FRASCO 100ML, ZOLPIDEM - ...
1  CRLN197704540  PARACETAMOL 10 MG/ML FRASCO 100ML, DOLANTINA 1...
2  CSBR193504540  FUROSEMIDA 20 mg ampollas 2 ml, LACTULOSA (DUP...
3  CSPR193305700  FARMACO NO INCLUIDO EN GUIA, OMEPRAZOL CAPSULA...
4  FDFD192912290  FARMACO NO INCLUIDO EN GUIA, HIBOR 3.500 UI je...


In [27]:
# Convertir a categorico
columnas_a_convertir = ['itipsexo', 'gcodipos','ireingre','gsitalta', 'gmotalta','gdiagalt','ds_izq_der','ds_turno','ds_vivo_alta','ds_dia_semana_llegada', 'ds_mes_llegada', 'ds_tipo_fractura', 'vive_solo', 'ds_HTA','ds_alergia_medicamentosa','ds_alergia_alimenticia','ds_diabetes','ds_otras_alergias', 'lugar_residencia','lugar_procedencia','destino_alta','ds_ITU','ds_acido_folico','ds_anemia','ds_vitamina_d','ds_obesidad','ds_osteoporosis','ds_insuficiencia_respiratoria','ds_infeccion_respiratoria','ds_insuficiencia_cardiaca','ds_deterioro_cognitivo','ds_insuficiencia_renal' ]
df[columnas_a_convertir] = df[columnas_a_convertir].astype('category')

In [29]:
df.to_csv('DATOS_PREPROCESADOS.csv', index=False)

In [30]:
df

,gidenpac,itipsexo,gcodipos,ireingre,gsitalta,gmotalta,gdiagalt,ds_izq_der,ds_turno,ds_edad,...,ds_vitamina_d,ds_obesidad,ds_osteoporosis,ds_insuficiencia_respiratoria,ds_infeccion_respiratoria,ds_insuficiencia_cardiaca,ds_deterioro_cognitivo,ds_insuficiencia_renal,ds_HTA,ds_diabetes
1,GRMN192606210,0,24328,0,1,1,S72.012A,0,1,98,...,0,0,0,1,1,1,0,1,0,0
2,BLGR192807170,0,24006,0,2,1,S72.012A,0,1,96,...,0,0,0,0,0,0,1,0,0,0
3,LFLL192212310,0,24240,0,2,1,S72.141A,1,1,101,...,1,0,0,0,0,0,1,0,0,1
4,DRCL192405020,0,24600,0,2,1,S72.012A,0,2,100,...,0,0,0,1,1,1,0,0,0,1
5,MNHR191911290,0,24007,0,2,1,S72.012A,0,2,104,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975,BZMR193509440,1,24010,0,2,1,S72.012A,0,2,89,...,0,0,0,0,0,0,0,0,1,1
976,LVGR193509470,1,24001,0,2,1,S72.012A,0,1,89,...,0,0,0,0,0,1,1,1,1,1
977,RDDM193509620,1,24358,0,2,1,S72.141A,1,1,89,...,0,0,0,0,0,0,0,0,0,0
978,CBCS194203680,1,24240,0,2,1,S72.141A,1,1,82,...,1,0,0,0,0,0,0,0,0,1
